In [ ]:
# import necessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 



import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator


# Build our Spark Session and Context
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext
spark, sc


from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import countDistinct, col

In [2]:
!python src/run.py data/sample_sub1.csv

2021-04-08 20:09:22,366 - reco-cs - DEBUG - using training ratings from data/training.csv
2021-04-08 20:09:22,368 - reco-cs - DEBUG - using requests from data/requests.csv
2021-04-08 20:09:22,368 - reco-cs - DEBUG - using output as data/sample_sub1.csv
2021-04-08 20:09:23,642 - reco-cs - DEBUG - starting fit
2021-04-08 20:09:23,642 - reco-cs - DEBUG - finishing fit
2021-04-08 20:09:23,642 - reco-cs - DEBUG - starting predict
2021-04-08 20:09:23,643 - reco-cs - DEBUG - request count: 200209
2021-04-08 20:09:23,682 - reco-cs - DEBUG - finishing predict


In [17]:
movies = pd.read_csv('data/movies.dat', sep='\t|::',header=None, engine='python')
users = pd.read_csv('data/users.dat', sep='\t|::',header=None, engine='python')
ratings = pd.read_csv('data/training.csv')

In [23]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [19]:
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [20]:
ratings

,user,movie,rating,timestamp
0,6040,858,4,956703932
1,6040,593,5,956703954
2,6040,2384,4,956703954
3,6040,1961,4,956703977
4,6040,2019,5,956703977
...,...,...,...,...
799995,1875,2940,5,975767255
799996,1875,589,4,975767289
799997,1875,110,4,975767289
799998,1875,2791,3,975767289


In [24]:
ratings = ratings.drop('timestamp', axis=1, inplace=True)

In [25]:
schema = StructType([
    StructField("user", IntegerType(), True),
    StructField("movie", IntegerType(), True),
    StructField("rating", IntegerType(), True)])

In [28]:
def load_data_to_spark():
    df = pd.read_csv('data/training.csv')
    df = df.drop(columns='timestamp')
    s_df = spark.createDataFrame(df)
    return s_df, df

In [ ]:
s_df, p_df = load_data_to_spark()

In [29]:
ratings

In [ ]:
train, test = ratings_df.randomSplit([0.8, 0.2], seed=1)